Bekannte Ausnahmen X - Achse: 18 18.1 18.2

In [11]:
import pandas as pd
import numpy as np


# Name/ Pfad zur Excel-Datei anpassen:
excel_file = 'Annex 3 (FINREP).xlsx'

# Alle Worksheet-Namen holen
xl = pd.ExcelFile(excel_file)

print(xl.sheet_names)

list_index_test = ['18.1', '18.2']
list_index_test = xl.sheet_names[1:]
print(list_index_test)

['Index', '1.1', '1.2', '1.3', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '18.1', '18.2', '19', '20', '21', '22', '23', '24', '25', '26', '30', '31', '40', '41', '42 ', '43', '44', '45', '46', '47']
['1.1', '1.2', '1.3', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '18.1', '18.2', '19', '20', '21', '22', '23', '24', '25', '26', '30', '31', '40', '41', '42 ', '43', '44', '45', '46', '47']


In [13]:


all_data = []

for sheet_name in list_index_test:
    # Wir lesen das komplette Sheet ohne Header-Interpretation, 
    # damit wir alles „roh“ haben:
    df = pd.read_excel(excel_file, sheet_name=sheet_name, header=None)
    
    # Suche nach einer Zelle, die "references" (case-insensitive) enthält.
    # Wir durchmustern alle Zellen des DataFrame df.
    # Dabei wandeln wir Werte zu Strings um und prüfen "contains('references')"
    references_mask = df.apply(lambda col: col.astype(str).str.contains('references', case=False, na=False))
    
    # Die True/False-Matrix flatten wir und filtern die True-Werte heraus:
    found_positions = np.where(references_mask.values == True)
    
    if len(found_positions[0]) == 0:
        # Keine Zelle mit "references" gefunden; nächstes Worksheet
        print(f"Keine Zelle mit 'References' in '{sheet_name}' gefunden. Überspringe dieses Sheet.")
        continue
    
    # Wir nehmen hier einfach die ERSTE gefundene Position:
    first_row = found_positions[0][0]
    first_col = found_positions[1][0]
    
    # Angenommen: Spalte mit "References" ist first_col.
    # Wir nehmen an, dass "Line" in (first_col - 2) und "Text" in (first_col - 1) stehen.
    line_col_idx = first_col - 2
    text_col_idx = first_col - 1

    if sheet_name == '18' or sheet_name == '18.1':
        print("test")
        line_col_idx = first_col - 3
        text_col_idx = first_col - 2

    if sheet_name == '18.2':
        line_col_idx = first_col - 3
        text_col_idx = first_col - 1
    
    # Prüfen, ob das auch valide Spalten sind
    if line_col_idx < 0 or text_col_idx < 0:
        print(f"Spalten für 'Line'/'Text' liegen außerhalb des Bereichs. Worksheet '{sheet_name}' wird übersprungen.")
        continue
    
    # Außerdem nehmen wir an, dass die Zellen ab der nächsten Zeile (first_row+1) unsere Daten enthalten.
    df_data = df.iloc[first_row-2:, :]
    
    # Jetzt iterieren wir über diese Datenzeilen und lesen "Line", "Text" und "LegalReference".
    # "Line" = df_data.iloc[:, line_col_idx]
    # "Text" = df_data.iloc[:, text_col_idx]
    # "LegalReference" = df_data.iloc[:, first_col]
    
    for i in range(len(df_data)):
        line_val  = df_data.iloc[i, line_col_idx]
        text_val  = df_data.iloc[i, text_col_idx]
        ref_val   = df_data.iloc[i, first_col]
        
        # Nur Zeilen übernehmen, die nicht alle Null/NaN sind
        if pd.notnull(line_val) or pd.notnull(text_val) or pd.notnull(ref_val):
            all_data.append({
                'Worksheet': sheet_name,
                'Line': line_val,
                'Text': text_val,
                'LegalReference': ref_val
            })

# Alle Daten in ein DataFrame
final_df = pd.DataFrame(all_data)
print(final_df)

# Optional: Speichern
final_df.to_csv('output_y.csv', index=False, encoding='utf-8')


test
test
Keine Zelle mit 'References' in '40' gefunden. Überspringe dieses Sheet.
     Worksheet        Line                                               Text  \
0          1.1  1.1 Assets                                                NaN   
1          1.1         NaN                                                NaN   
2          1.1        0010  Cash, cash balances at central banks and other...   
3          1.1        0020                                       Cash on hand   
4          1.1        0030                     Cash balances at central banks   
...        ...         ...                                                ...   
1831        47         NaN                                                NaN   
1832        47        0010  Non-performing loans and advances: weighted av...   
1833        47        0020  Net cumulated recoveries from litigation proce...   
1834        47        0030  Gross carrying amount reduction from litigatio...   
1835        47        0040